In [173]:
import pandas as pd
import numpy as np


In [174]:
### data inladen
wedstrijden = pd.read_excel('../../Data/Silver/wedstrijden_cleaned.xlsx')
corners = pd.read_excel('../../Data/Silver/cornerballen_cleaned.xlsx')

merged_df = pd.merge(wedstrijden, corners, on='wedstrijd_id', how='inner')


In [175]:
# voor merged_df gaan we alle kolommen met een v in opsplitsen in thuis_kolom en uit_kolom
merged_df[['klassement_thuis', 'klassement_uit']] = merged_df['klassement_uitslag'].str.split('v', expand=True)
merged_df[['uitslag_thuis', 'uitslag_uit']] = merged_df['uitslag'].str.split('v', expand=True)
merged_df[['score_voor_thuis', 'score_voor_uit']] = merged_df['score_voor'].str.split('v', expand=True)

# Convert to numeric
cols_to_convert = [
    'klassement_thuis', 'klassement_uit',
    'uitslag_thuis', 'uitslag_uit',
    'score_voor_thuis', 'score_voor_uit'
]

for col in cols_to_convert:
    merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce').astype('Int64')

# drop de kolommen die we niet meer nodig hebben
merged_df = merged_df.drop(columns=['klassement_uitslag', 'uitslag', 'score_voor'])


In [176]:
merged_df.dtypes

wedstrijd_id                int64
datum                      object
niveau                     object
corners_uitslag            object
schepcorner_id              int64
ploegnaam                  object
ervaring_schepper         float64
ervaring_kopper           float64
ervaring_duo              float64
is_thuisploeg               int64
kwart                       int64
is_schepper_bank            int64
is_kopper_bank              int64
schepper_pos_rechts         int64
positie                     int64
kop_verplaatsen_achter      int64
kop_veel_verplaatsen        int64
kop_hor_verplaatsen         int64
goal                        int64
klassement_thuis            Int64
klassement_uit              Int64
uitslag_thuis               Int64
uitslag_uit                 Int64
score_voor_thuis            Int64
score_voor_uit              Int64
dtype: object

In [177]:
wedstrijden.head()

,wedstrijd_id,datum,niveau,uitslag,corners_uitslag,klassement_uitslag
0,1,2025-03-15 00:00:00,4,4v1,2v1,1v5
1,2,2024-09-03 00:00:00,4,11v2,2v1,8v3
2,3,2024-09-03 00:00:00,4,4v6,1v1,3v10
3,4,2024-09-03 00:00:00,3,5v3,1v3,4v9
4,5,2024-09-03 00:00:00,4,7v2,0v2,NaN


In [178]:
corners.head()

,schepcorner_id,wedstrijd_id,ploegnaam,ervaring_schepper,ervaring_kopper,ervaring_duo,is_thuisploeg,kwart,is_schepper_bank,is_kopper_bank,score_voor,schepper_pos_rechts,positie,kop_verplaatsen_achter,kop_veel_verplaatsen,kop_hor_verplaatsen,goal
0,1,1,marathon,4.0,4.0,4.0,1,1,1,0,0v0,1,5,0,0,0,1
1,2,1,marathon,4.0,4.0,4.0,1,2,0,0,2v0,1,5,0,0,0,0
2,3,1,poba juniors,3.0,2.0,3.0,0,3,0,0,2v0,0,2,0,0,1,0
3,4,2,mvc rudie,4.0,5.0,4.0,1,3,1,0,4v1,1,5,0,1,1,0
4,5,2,uncle abes patty pounders,1.0,5.0,1.0,0,3,1,0,5v2,1,5,0,0,0,0


### data exploration

In [179]:
### missing values in de merged_df
print(merged_df.isnull().sum()/len(merged_df)*100)

wedstrijd_id               0.000000
datum                      0.000000
niveau                     0.000000
corners_uitslag           34.328358
schepcorner_id             0.000000
ploegnaam                  0.000000
ervaring_schepper         29.519071
ervaring_kopper           29.850746
ervaring_duo              29.519071
is_thuisploeg              0.000000
kwart                      0.000000
is_schepper_bank           0.000000
is_kopper_bank             0.000000
schepper_pos_rechts        0.000000
positie                    0.000000
kop_verplaatsen_achter     0.000000
kop_veel_verplaatsen       0.000000
kop_hor_verplaatsen        0.000000
goal                       0.000000
klassement_thuis          41.956882
klassement_uit            41.956882
uitslag_thuis              0.000000
uitslag_uit                0.000000
score_voor_thuis           0.000000
score_voor_uit             0.000000
dtype: float64


In [180]:
print(wedstrijden.isna().sum()/len(merged_df)*100)

wedstrijd_id           0.000000
datum                  0.000000
niveau                 0.000000
uitslag                0.000000
corners_uitslag       12.935323
klassement_uitslag    16.749585
dtype: float64


In [181]:
print(corners.isna().sum()/len(merged_df)*100)

schepcorner_id             0.000000
wedstrijd_id               0.000000
ploegnaam                  0.000000
ervaring_schepper         29.519071
ervaring_kopper           29.850746
ervaring_duo              29.519071
is_thuisploeg              0.000000
kwart                      0.000000
is_schepper_bank           0.000000
is_kopper_bank             0.000000
score_voor                 0.000000
schepper_pos_rechts        0.000000
positie                    0.000000
kop_verplaatsen_achter     0.000000
kop_veel_verplaatsen       0.000000
kop_hor_verplaatsen        0.000000
goal                       0.000000
dtype: float64


In [182]:
# aggregate niveaus, national, kern_hoog, kern_laag
merged_df["niveau"] = merged_df["niveau"].replace({"n1": "nationaal", "n4": "nationaal","p2": "nationaal", 1: "kern_hoog", 2: "kern_hoog", 3: "kern_laag", 4: "kern_laag"})

In [183]:
### veel missing values bij ervaring, hoe imputen?
# 1 imputen met vorige wedstrijden, anders met gemiddelde van niveau
# 2 prediction model bouwen

### missing values in corners_uistlag, geen probleem wordt niet gebruikt in de analyse
# kolom droppen

### klassement uitslag missing values, hoe imputen?
# 1 imputen met vorige wedstrijden, of gemiddelde afstand gebruiken
# 2 anders met finale uitslag van klassement, nadeel veel manueel werk

In [184]:
# droppen corners_uitslag kolom
merged_df.drop(columns=['corners_uitslag'], inplace=True)


In [185]:
merged_df.head()

,wedstrijd_id,datum,niveau,schepcorner_id,ploegnaam,ervaring_schepper,ervaring_kopper,ervaring_duo,is_thuisploeg,kwart,...,kop_verplaatsen_achter,kop_veel_verplaatsen,kop_hor_verplaatsen,goal,klassement_thuis,klassement_uit,uitslag_thuis,uitslag_uit,score_voor_thuis,score_voor_uit
0,1,2025-03-15 00:00:00,kern_laag,1,marathon,4.0,4.0,4.0,1,1,...,0,0,0,1,1,5,4,1,0,0
1,1,2025-03-15 00:00:00,kern_laag,2,marathon,4.0,4.0,4.0,1,2,...,0,0,0,0,1,5,4,1,2,0
2,1,2025-03-15 00:00:00,kern_laag,3,poba juniors,3.0,2.0,3.0,0,3,...,0,0,1,0,1,5,4,1,2,0
3,2,2024-09-03 00:00:00,kern_laag,4,mvc rudie,4.0,5.0,4.0,1,3,...,0,1,1,0,8,3,11,2,4,1
4,2,2024-09-03 00:00:00,kern_laag,5,uncle abes patty pounders,1.0,5.0,1.0,0,3,...,0,0,0,0,8,3,11,2,5,2


In [186]:
# Imputatiefunctie
def imputatie_rij(rij, ervaring_col, ploegnaam_means, niveau_means):
    if pd.notnull(rij[ervaring_col]):
        return rij[ervaring_col]
    ploegnaam_mean = ploegnaam_means.get(rij['ploegnaam'], np.nan)
    if pd.notnull(ploegnaam_mean):
        return ploegnaam_mean
    niveau_mean = niveau_means.get(rij['niveau'], np.nan)
    return niveau_mean

In [187]:
# imputen van ervaring
def impute_ervaring(df, ervaring_col):
    """
    imputen met gemiddelde vorige wedstrijden van ploeg, anders met gemiddelde van niveau
    """

    df = df.copy()

    ploegnaam_means = df.groupby('ploegnaam')[ervaring_col].mean()
    niveau_means = df.groupby('niveau')[ervaring_col].mean()

    df[ervaring_col] = df.apply(
        imputatie_rij,
        axis=1,
        ervaring_col=ervaring_col,
        ploegnaam_means=ploegnaam_means,
        niveau_means=niveau_means
    )

    return df

In [188]:
ervaring_imputed_df = impute_ervaring(merged_df, 'ervaring_schepper')
ervaring_imputed_df = impute_ervaring(ervaring_imputed_df, 'ervaring_kopper')
ervaring_imputed_df = impute_ervaring(ervaring_imputed_df, 'ervaring_duo')


In [189]:
print(ervaring_imputed_df.isnull().sum()/len(merged_df)*100)

wedstrijd_id               0.000000
datum                      0.000000
niveau                     0.000000
schepcorner_id             0.000000
ploegnaam                  0.000000
ervaring_schepper          0.000000
ervaring_kopper            0.000000
ervaring_duo               0.000000
is_thuisploeg              0.000000
kwart                      0.000000
is_schepper_bank           0.000000
is_kopper_bank             0.000000
schepper_pos_rechts        0.000000
positie                    0.000000
kop_verplaatsen_achter     0.000000
kop_veel_verplaatsen       0.000000
kop_hor_verplaatsen        0.000000
goal                       0.000000
klassement_thuis          41.956882
klassement_uit            41.956882
uitslag_thuis              0.000000
uitslag_uit                0.000000
score_voor_thuis           0.000000
score_voor_uit             0.000000
dtype: float64


In [190]:
ervaring_imputed_df[ervaring_imputed_df['ervaring_schepper'].isnull()]

,wedstrijd_id,datum,niveau,schepcorner_id,ploegnaam,ervaring_schepper,ervaring_kopper,ervaring_duo,is_thuisploeg,kwart,...,kop_verplaatsen_achter,kop_veel_verplaatsen,kop_hor_verplaatsen,goal,klassement_thuis,klassement_uit,uitslag_thuis,uitslag_uit,score_voor_thuis,score_voor_uit


In [191]:
### TODO: klassement imputeren
# eerst klassement_uitslag opsplitsen in thuis_klassement en uit_kassement
ervaring_imputed_df[["ploegnaam", "datum",'klassement_thuis', 'klassement_uit', "is_thuisploeg"]]

,ploegnaam,datum,klassement_thuis,klassement_uit,is_thuisploeg
0,marathon,2025-03-15 00:00:00,1,5,1
1,marathon,2025-03-15 00:00:00,1,5,1
2,poba juniors,2025-03-15 00:00:00,1,5,0
3,mvc rudie,2024-09-03 00:00:00,8,3,1
4,uncle abes patty pounders,2024-09-03 00:00:00,8,3,0
...,...,...,...,...,...
598,sint gillis waas,2024-11-29 00:00:00,10,12,0
599,erwetegem,2024-12-06 00:00:00,5,2,0
600,erwetegem,2024-12-06 00:00:00,5,2,0
601,erwetegem,2024-12-06 00:00:00,5,2,0


In [192]:
ervaring_imputed_df.shape

(603, 24)

### Feature Engineering

In [193]:
basetable = ervaring_imputed_df.copy()

In [194]:
# klassement positie 
basetable["ranking_klassement"] = np.where(basetable["is_thuisploeg"] == 1, basetable["klassement_thuis"], basetable["klassement_uit"])
basetable["klassement_veschil"] = abs(basetable["klassement_thuis"] - basetable["klassement_uit"])



In [195]:
#score_verschil_voor
basetable["score_verschil_voor"] = abs(basetable["score_voor_thuis"] - basetable["score_voor_uit"])

In [196]:
# de hoeveelste schepcorner van de wedstrijd
basetable["schepcorner_nummer"] = basetable.groupby("wedstrijd_id").cumcount() + 1

In [197]:
basetable["schepcorner_nummer"].unique()

array([1, 2, 3, 4, 5, 6], dtype=int64)

In [198]:
basetable.columns

Index(['wedstrijd_id', 'datum', 'niveau', 'schepcorner_id', 'ploegnaam',
       'ervaring_schepper', 'ervaring_kopper', 'ervaring_duo', 'is_thuisploeg',
       'kwart', 'is_schepper_bank', 'is_kopper_bank', 'schepper_pos_rechts',
       'positie', 'kop_verplaatsen_achter', 'kop_veel_verplaatsen',
       'kop_hor_verplaatsen', 'goal', 'klassement_thuis', 'klassement_uit',
       'uitslag_thuis', 'uitslag_uit', 'score_voor_thuis', 'score_voor_uit',
       'ranking_klassement', 'klassement_veschil', 'score_verschil_voor',
       'schepcorner_nummer'],
      dtype='object')

In [199]:
columns_to_drop = ["wedstrijd_id"]
basetable.drop(columns=columns_to_drop, inplace=True)

In [200]:
# schrijf weg
basetable.to_csv('../../Data/Gold/basetable.csv', index=False)